In [1]:
import cv2
import numpy as np

In [ ]:
cap = cv2.VideoCapture(0)

ret, frame1 = cap.read()

prvsImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)

hsv_mask[:,:,1] = 255

while True:
    
    ret, frame2 = cap.read()
    
    nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Calculate dense optical flow with Farneback
    flow = cv2.calcOpticalFlowFarneback(prvsImg, nextImg, None, pyr_scale = 0.5, levels = 5, winsize = 11, iterations = 5, poly_n = 5, poly_sigma = 1.1, flags = 0)
    
    # Compute the magnitude and angle of the 2D vectors: x vector and y vector
    magnitude, angle = cv2.cartToPolar(flow[:,:,0], flow[:,:,1], angleInDegrees=True)
    
    # Set image hue according to the optical flow direction
    hsv_mask[:,:,0] = angle
        
    # Set image value according to the optical flow magnitude (normalized)
    hsv_mask[:,:,2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    
    # Convert HSV to RGB (BGR) color representation
    rgb = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    
    # Open a new window and displays the output frame
    dense_flow = cv2.addWeighted(frame2, 1,rgb, 1, 0)
    cv2.imshow("Dense optical flow", dense_flow)
    
    # Update previous frame
    prvsImg = nextImg
    
    # Frame are read by intervals of 10 millisecond. The programs breaks out of the while loop when the user presses the ‘q’ key
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()